<a href="https://colab.research.google.com/github/Gaurav-phatkare/Retrieval-Augmented-Generation_RAG/blob/main/RAG_Fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!nvidia-smi

Sat Jul 27 17:16:33 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8              12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch

## building

1. Open a PDF document
2. Format the text of that PDF suitable for embedding model
3. embed into chunks and then convert it into numerical representation
4. use vector db to search relevent cuunk
5. crete prompt that incoporates the retrived peices of text
6. generate answer

## Import PDF


In [4]:
import os
import requests

pdf_path = "human-nutrition-text.pdf"

# download PDF

if not os.path.exists(pdf_path):
  print("File doesnt exists")

  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  filename = pdf_path
  res = requests.get(url)

  if res.status_code == 200:
    with open(filename, "wb") as file:
      file.write(res.content)
    print(f"file has been downloaded and stored as {filename}")

  else:
    print(f"failed to download file, Status code : {res.status_code}")
else:
  print("file exists")

file exists


In [5]:
!pip install pyMuPDF

In [6]:
!pip install tqdm

In [7]:
import fitz
from tqdm.auto import tqdm


def text_formatter(text:str):
  cleaned_text = text.replace("\n", " ").strip()
  return cleaned_text


def open_and_read_pdf(pdf_path):
  doc = fitz.open(pdf_path)

  pages_and_texts = []

  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()

    text = text_formatter(text=text)

    pages_and_texts.append({"page_number" : page_number - 41,
                            "page_char_count" : len(text),
                            "page_word_count" : len(text.split(" ")),
                            "page_sentence_count_raw" : len(text.split(". ")),
                            "page_token_count": len(text) / 4,
                            "text" : text})

  return pages_and_texts


pages_and_texts = open_and_read_pdf(pdf_path)
pages_and_texts[:3]




0it [00:00, ?it/s]

[{'page_number': -41,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count_raw': 1,
  'page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_number': -40,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count_raw': 1,
  'page_token_count': 0.0,
  'text': ''},
 {'page_number': -39,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count_raw': 1,
  'page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [8]:
import random

random.sample(pages_and_texts, 2)

[{'page_number': 622,
  'page_char_count': 1647,
  'page_word_count': 283,
  'page_sentence_count_raw': 12,
  'page_token_count': 411.75,
  'text': 'Diet, Supplements, and Chelated Supplements  In general, calcium supplements perform to a lesser degree than  dietary sources of calcium in providing many of the health benefits  linked to higher calcium intake. This is partly attributed to the fact  that dietary sources of calcium supply additional nutrients with  health-promoting activities. It is reported that chelated forms of  calcium supplements are easier to absorb as the chelation process  protects the calcium from oxalates and phytates that may bind with  the calcium in the intestines. However, these are more expensive  supplements and only increase calcium absorption up to 10 percent.  In people with low dietary intakes of calcium, calcium supplements  have a negligible benefit on bone health in the absence of a vitamin  D supplement. However, when calcium supplements are taken a

In [9]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition
1,-40,0,1,1,0.00,
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [10]:
df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593
std,348.86387,560.441673,95.747365,6.187226,140.110418
min,-41.00000,0.000000,1.000000,1.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500
50%,562.50000,1232.500000,215.000000,10.000000,308.125000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500
max,1166.00000,2308.000000,429.000000,32.000000,577.000000


In [11]:
# Text processing
from spacy.lang.en import English

nlp = English()


nlp.add_pipe('sentencizer')


doc = nlp("this is a sentence. This is another sentence... . I love tensorflow.")

# assert len(list(doc.sents)) == 3

list(doc.sents)

[this is a sentence., This is another sentence... ., I love tensorflow.]

In [12]:
for item in tqdm(pages_and_texts):
  item['sentences'] =  list(nlp(item['text']).sents)

  item['sentences'] = [str(sentence) for sentence in item['sentences']]

  item['page_sentence_count_spacy'] = len(item['sentences'])



  0%|          | 0/1208 [00:00<?, ?it/s]

In [13]:
random.sample(pages_and_texts, 1)

[{'page_number': 229,
  'page_char_count': 305,
  'page_word_count': 66,
  'page_sentence_count_raw': 1,
  'page_token_count': 76.25,
  'text': 'Two  Breadfruit by  Michael  Coghlan /  CC BY-SA 2.0  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  E ʻai i ka mea i loaʻa  What you have, eat  Learning Objectives  By the end of this chapter, you will be able to:  Introduction  |  229',
  'sentences': ['Two  Breadfruit by  Michael  Coghlan /  CC BY-SA 2.0  Introduction  UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM AND HUMAN NUTRITION PROGRAM  E ʻai i ka mea i loaʻa  What you have, eat  Learning Objectives  By the end of this chapter, you will be able to:  Introduction  |  229'],
  'page_sentence_count_spacy': 1}]

In [14]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000


In [15]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1
1,-40,0,1,1,0.00,,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2


In [16]:
chunk_size = 10

def split_list(input_list, slice_size = chunk_size):

  return [input_list[i:i+slice_size] for i in range(0, len(input_list), slice_size)]

test_list = list(range(25))

split_list(test_list)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
 [20, 21, 22, 23, 24]]

In [17]:
for item in tqdm(pages_and_texts):
  item['sentence_chunks'] = split_list(input_list = item['sentences'], slice_size = chunk_size)

  item['num_chunks'] = len(item['sentence_chunks'])

  0%|          | 0/1208 [00:00<?, ?it/s]

In [18]:
random.sample(pages_and_texts, 1)

[{'page_number': 376,
  'page_char_count': 795,
  'page_word_count': 141,
  'page_sentence_count_raw': 4,
  'page_token_count': 198.75,
  'text': 'When a  protein is  exposed to a  different  environment , such as  increased  temperature,  it unfolds  into a single  strand of  amino acids.  Figure 6.6 Protein Denaturation  \xa0 Learning Activities  Technology Note: The second edition of the Human  Nutrition Open Educational Resource (OER) textbook  features interactive learning activities.\xa0 These activities are  available in the web-based textbook and not available in the  downloadable versions (EPUB, Digital PDF, Print_PDF, or  Open Document).  Learning activities may be used across various mobile  devices, however, for the best user experience it is strongly  recommended that users complete these activities using a  desktop or laptop computer and in Google Chrome.  \xa0 376  |  The Role of Proteins in Foods: Cooking and Denaturation',
  'sentences': ['When a  protein is  exposed t

In [19]:
df = pd.DataFrame(pages_and_texts)

df.describe()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,1208.00000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000,1208.000000
mean,562.50000,1148.594371,198.889901,9.972682,287.148593,10.319536,1.525662
std,348.86387,560.441673,95.747365,6.187226,140.110418,6.300843,0.644397
min,-41.00000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,260.75000,762.750000,134.000000,4.000000,190.687500,5.000000,1.000000
50%,562.50000,1232.500000,215.000000,10.000000,308.125000,10.000000,1.000000
75%,864.25000,1605.250000,271.250000,14.000000,401.312500,15.000000,2.000000
max,1166.00000,2308.000000,429.000000,32.000000,577.000000,28.000000,3.000000


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   page_number                1208 non-null   int64  
 1   page_char_count            1208 non-null   int64  
 2   page_word_count            1208 non-null   int64  
 3   page_sentence_count_raw    1208 non-null   int64  
 4   page_token_count           1208 non-null   float64
 5   text                       1208 non-null   object 
 6   sentences                  1208 non-null   object 
 7   page_sentence_count_spacy  1208 non-null   int64  
 8   sentence_chunks            1208 non-null   object 
 9   num_chunks                 1208 non-null   int64  
dtypes: float64(1), int64(6), object(3)
memory usage: 94.5+ KB


In [21]:
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text,sentences,page_sentence_count_spacy,sentence_chunks,num_chunks
0,-41,29,4,1,7.25,Human Nutrition: 2020 Edition,[Human Nutrition: 2020 Edition],1,[[Human Nutrition: 2020 Edition]],1
1,-40,0,1,1,0.00,,[],0,[],0
2,-39,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...,[Human Nutrition: 2020 Edition UNIVERSITY OF...,1,[[Human Nutrition: 2020 Edition UNIVERSITY O...,1
3,-38,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...,[Human Nutrition: 2020 Edition by University o...,1,[[Human Nutrition: 2020 Edition by University ...,1
4,-37,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...,[Contents Preface University of Hawai‘i at M...,2,[[Contents Preface University of Hawai‘i at ...,1


In [22]:
# splitting each chunk into its own item

import re

pages_and_chunks = []

for item in tqdm(pages_and_texts):
  for sentence_c in item['sentence_chunks']:
    chunk_dict = {}

    chunk_dict['page_number'] = item['page_number']

    joined_sent_chunk = "".join(sentence_c).replace("  ", " ").strip()
    joined_sent_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sent_chunk)

    chunk_dict['sentence_chunk'] = joined_sent_chunk

    chunk_dict['chunk_char_count'] = len(joined_sent_chunk)

    chunk_dict['chunk_word_count'] = len([word for word in joined_sent_chunk.split(" ")])
    chunk_dict['chunk_token_count'] = len(joined_sent_chunk) / 4

    pages_and_chunks.append(chunk_dict)

  0%|          | 0/1208 [00:00<?, ?it/s]

In [23]:
len(pages_and_chunks)

1843

In [24]:
random.sample(pages_and_chunks, 1)

[{'page_number': 894,
  'sentence_chunk': '894 | Late Adolescence',
  'chunk_char_count': 22,
  'chunk_word_count': 4,
  'chunk_token_count': 5.5}]

In [25]:
df = pd.DataFrame(pages_and_chunks)

df.describe()

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,1843.000000,1843.000000,1843.000000,1843.000000
mean,583.381443,734.829626,112.720022,183.707406
std,347.788670,447.431540,71.065283,111.857885
min,-41.000000,12.000000,3.000000,3.000000
25%,280.500000,315.000000,45.000000,78.750000
50%,586.000000,746.000000,114.000000,186.500000
75%,890.000000,1118.500000,173.000000,279.625000
max,1166.000000,1831.000000,297.000000,457.750000


In [26]:
# filter out chunks of text for short chunks

min_token_len = 30

for row in df[df['chunk_token_count'] <= min_token_len].sample(5).iterrows():
  print(f"Chunk token count : {row[1]['chunk_token_count']} | Text : {row[1]['sentence_chunk']}")

Chunk token count : 12.5 | Text : Figure 11.2 The Structure of Hemoglobin Iron | 655
Chunk token count : 3.25 | Text : 814 | Infancy
Chunk token count : 28.75 | Text : Image by FDA/ Changes to the Nutrition Facts Label Figure 12.5 Food Serving Sizes 728 | Discovering Nutrition Facts
Chunk token count : 9.25 | Text : 490 | Factors Affecting Energy Intake
Chunk token count : 4.25 | Text : Introduction | 61


In [27]:
pages_and_chunks_over_min_token_len = df[df['chunk_token_count'] > min_token_len].to_dict(orient = "records")

pages_and_chunks_over_min_token_len[:2]



[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [28]:
random.sample(pages_and_chunks_over_min_token_len, 1)

[{'page_number': 842,
  'sentence_chunk': 'Pocket guide to pediatric nutrition assessment, 2nd ed. Academy of Nutrition and Dietetics. NOTE: These are general guidelines for the healthy, full-term infant per day; serving sizes may vary with individual infants. Start complementary foods when developmentally ready, about 6 months; start with about 0.5-1 ounce. 842 | Infancy',
  'chunk_char_count': 331,
  'chunk_word_count': 49,
  'chunk_token_count': 82.75}]

In [29]:
## Embedding text chunks
# convert it into meaningful numerical representations
!pip install sentence_transformers


In [30]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path = "all-mpnet-base-v2",
                                       device = 'cpu')

sentences = ["The is first sentence",
             "I love tensorflow",
             "Tensorflow is used to create deep learning models"]

embeddings = embedding_model.encode(sentences)

embeddings_dict = dict(zip(sentences, embeddings))

for s, e in embeddings_dict.items():
  print(f"Sentence : {s}")
  print(f"Embedding: {e}")
  print("")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Sentence : The is first sentence
Embedding: [ 7.31820092e-02 -1.61321536e-02 -1.15740197e-02  2.08527446e-02
 -6.85733557e-02  4.06724364e-02 -2.10723169e-02  2.47126687e-02
 -4.70292680e-02 -7.26543739e-03  1.52472192e-02 -3.00663076e-02
  6.55668154e-02 -2.35591270e-02  1.30877784e-02 -2.87996121e-02
  1.62504558e-02  2.79548187e-02  2.96473727e-02 -6.83668815e-03
  2.95617208e-02  3.22184414e-02  1.40264733e-02  1.94812883e-02
  7.42335152e-03 -1.20363701e-02  2.05279533e-02  4.25696336e-02
 -1.84285529e-02 -5.15345521e-02  9.57240351e-03  1.14023760e-02
 -2.68759672e-03 -3.12451944e-02  1.76468416e-06 -8.07849038e-03
 -3.66575941e-02 -3.02734561e-02 -1.20519511e-02  6.74865514e-05
 -2.28438340e-02  5.51525652e-02  1.17068607e-02  3.52899469e-02
 -1.90833863e-02  2.74438281e-02  2.06340905e-02  6.54786825e-02
 -1.78782362e-02  2.59095263e-02 -2.16650646e-02 -1.81301516e-02
 -4.60181050e-02 -1.37205590e-02  9.41626058e-06 -2.37068161e-02
  2.66071297e-02 -3.42558250e-02 -4.00836021e-

In [31]:
len(embeddings[0])

768

In [32]:
pages_and_chunks_over_min_token_len[:2]

[{'page_number': -39,
  'sentence_chunk': 'Human Nutrition: 2020 Edition UNIVERSITY OF HAWAI‘I AT MĀNOA FOOD SCIENCE AND HUMAN NUTRITION PROGRAM ALAN TITCHENAL, SKYLAR HARA, NOEMI ARCEO CAACBAY, WILLIAM MEINKE-LAU, YA-YUN YANG, MARIE KAINOA FIALKOWSKI REVILLA, JENNIFER DRAPER, GEMADY LANGFELDER, CHERYL GIBBY, CHYNA NICOLE CHUN, AND ALLISON CALABRESE',
  'chunk_char_count': 308,
  'chunk_word_count': 42,
  'chunk_token_count': 77.0},
 {'page_number': -38,
  'sentence_chunk': 'Human Nutrition: 2020 Edition by University of Hawai‘i at Mānoa Food Science and Human Nutrition Program is licensed under a Creative Commons Attribution 4.0 International License, except where otherwise noted.',
  'chunk_char_count': 210,
  'chunk_word_count': 30,
  'chunk_token_count': 52.5}]

In [33]:
# %%time

# embedding_model.to('cpu')

# for i in tqdm(pages_and_chunks_over_min_token_len):
#   i['embedding'] = embedding_model.encode(i['sentence_chunk'])

In [34]:
%%time
embedding_model.to('cuda')

for i in tqdm(pages_and_chunks_over_min_token_len):
  i['embedding'] = embedding_model.encode(i['sentence_chunk'])

  0%|          | 0/1680 [00:00<?, ?it/s]

CPU times: user 34.5 s, sys: 442 ms, total: 34.9 s
Wall time: 43 s


In [35]:
%%time

text_chunks = [i['sentence_chunk'] for i in pages_and_chunks_over_min_token_len]

text_chunks[419]

CPU times: user 582 µs, sys: 0 ns, total: 582 µs
Wall time: 588 µs


'often. • Calm your “sweet tooth” by eating fruits, such as berries or an apple. • Replace sugary soft drinks with seltzer water, tea, or a small amount of 100 percent fruit juice added to water or soda water. The Food Industry: Functional Attributes of Carbohydrates and the Use of Sugar Substitutes In the food industry, both fast-releasing and slow-releasing carbohydrates are utilized to give foods a wide spectrum of functional attributes, including increased sweetness, viscosity, bulk, coating ability, solubility, consistency, texture, body, and browning capacity. The differences in chemical structure between the different carbohydrates confer their varied functional uses in foods. Starches, gums, and pectins are used as thickening agents in making jam, cakes, cookies, noodles, canned products, imitation cheeses, and a variety of other foods. Molecular gastronomists use slow- releasing carbohydrates, such as alginate, to give shape and texture to their fascinating food creations. Add

In [36]:
len(text_chunks)

1680

In [37]:
%%time

text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size = 32,
                                               convert_to_tensor = True)

text_chunk_embeddings



CPU times: user 24.9 s, sys: 51.4 ms, total: 24.9 s
Wall time: 24.5 s


tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [38]:
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)

embeddings_df_save_path = 'text_chunks_and_embeddings_df.csv'
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index = False)

In [39]:
text_embeddings_df_load = pd.read_csv(embeddings_df_save_path)
text_embeddings_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,[ 6.74242675e-02 9.02281404e-02 -5.09548886e-...
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,[ 5.52156419e-02 5.92139773e-02 -1.66167244e-...
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,[ 2.79801842e-02 3.39813754e-02 -2.06426680e-...
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,[ 6.82566911e-02 3.81275006e-02 -8.46854132e-...
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,[ 3.30264494e-02 -8.49763490e-03 9.57159605e-...


In [40]:
import random
import torch
import numpy as np
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

text_chunks_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

text_chunks_embedding_df['embedding'] = text_chunks_embedding_df['embedding'].apply(lambda x: np.fromstring(x.strip('[]'), sep = " "))


embeddings = torch.tensor(np.stack(text_chunks_embedding_df['embedding'].tolist(), axis = 0), dtype = torch.float32).to(device)

pages_and_chunks = text_chunks_and_embeddings_df.to_dict(orient = 'records')



In [41]:
text_chunks_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,-39,Human Nutrition: 2020 Edition UNIVERSITY OF HA...,308,42,77.00,"[0.0674242675, 0.0902281404, -0.00509548886, -..."
1,-38,Human Nutrition: 2020 Edition by University of...,210,30,52.50,"[0.0552156419, 0.0592139773, -0.0166167244, -0..."
2,-37,Contents Preface University of Hawai‘i at Māno...,766,114,191.50,"[0.0279801842, 0.0339813754, -0.020642668, 0.0..."
3,-36,Lifestyles and Nutrition University of Hawai‘i...,941,142,235.25,"[0.0682566911, 0.0381275006, -0.00846854132, -..."
4,-35,The Cardiovascular System University of Hawai‘...,998,152,249.50,"[0.0330264494, -0.0084976349, 0.00957159605, -..."


In [42]:
# embeddings = text_chunks_embedding_df['embedding'].tolist()
# embeddings
embeddings

tensor([[ 0.0674,  0.0902, -0.0051,  ..., -0.0221, -0.0232,  0.0126],
        [ 0.0552,  0.0592, -0.0166,  ..., -0.0120, -0.0103,  0.0227],
        [ 0.0280,  0.0340, -0.0206,  ..., -0.0054,  0.0213,  0.0313],
        ...,
        [ 0.0771,  0.0098, -0.0122,  ..., -0.0409, -0.0752, -0.0241],
        [ 0.1030, -0.0165,  0.0083,  ..., -0.0574, -0.0283, -0.0295],
        [ 0.0864, -0.0125, -0.0113,  ..., -0.0522, -0.0337, -0.0299]],
       device='cuda:0')

In [43]:
embeddings.shape

torch.Size([1680, 768])

In [44]:
from sentence_transformers import util, SentenceTransformer

embedding_model = SentenceTransformer(model_name_or_path='all-mpnet-base-v2',
                                      device=device)

###Embedding model ready
Small semantic search pipeline

In [48]:
query = 'breastfeeding timeline for infants'
print(f'Query : {query}')


query_embedding = embedding_model.encode(query, convert_to_tensor = True).to('cuda')


from time import perf_counter as timer
start_time = timer()
dot_scores = util.dot_score(query_embedding, embeddings)[0]
end_time = timer()

print(f"Time taken to get score on {len(embeddings)} embeddings : {end_time - start_time:.5f} seconds")

print("")

top_results_ = torch.topk(dot_scores, 5)

top_results_

Query : breastfeeding timeline for infants
Time taken to get score on 1680 embeddings : 0.00021 seconds



torch.return_types.topk(
values=tensor([0.6749, 0.6574, 0.6543, 0.6354, 0.6168], device='cuda:0'),
indices=tensor([1151, 1169, 1160, 1144, 1138], device='cuda:0'))

In [49]:
pages_and_chunks[1151]['sentence_chunk'], pages_and_chunks[1151]['page_number']

('milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant becomes more efficient, and the number of feedings per day often become fewer even though the amount of milk consumed stays the same. After about six months, infants can gradually begin to consume solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition. Complementary foods include baby meats, vegetables, fruits, infant cereal, and dairy products such as yogurt, but not infant formula. Infant formula is a substitute, not a complement to breastmilk. In addition to complementary foods, the World Health Organization recommends that breastfeeding continue up to 2 

In [50]:
larger_embeddings = torch.randn(100*embeddings.shape[0], 768).to(device)

print(f'Embeddings shape: {larger_embeddings.shape}')

start_time = timer()
dot_scores = util.dot_score(query_embedding, larger_embeddings)[0]
end_time = timer()



print(f"Time taken to get score on {len(larger_embeddings)} embeddings : {end_time - start_time:.5f} seconds")

print("")



Embeddings shape: torch.Size([168000, 768])
Time taken to get score on 168000 embeddings : 0.00104 seconds



In [57]:
import textwrap

def print_wrapped(text, wrap_length = 100):
  wrapped_text = textwrap.fill(text, wrap_length)
  print(wrapped_text)

In [58]:
print(f"Query : {query}")
print("")

print("Results: ")

for s, i in zip(top_results_[0], top_results_[1]):
  print(f'Score: {s:.4f}')
  print('Text:')
  print_wrapped(pages_and_chunks[i]['sentence_chunk'])
  print(f'Page number: {pages_and_chunks[i]["page_number"]}')
  print("_____________________________________________________")

Query : breastfeeding timeline for infants

Results: 
Score: 0.6749
Text:
milk is the best source to fulfill nutritional requirements. An exclusively breastfed infant does
not even need extra water, including in hot climates. A newborn infant (birth to 28 days) requires
feedings eight to twelve times a day or more. Between 1 and 3 months of age, the breastfed infant
becomes more efficient, and the number of feedings per day often become fewer even though the amount
of milk consumed stays the same. After about six months, infants can gradually begin to consume
solid foods to help meet nutrient needs. Foods that are added in addition to breastmilk are called
complementary foods. Complementary foods should be nutrient dense to provide optimal nutrition.
Complementary foods include baby meats, vegetables, fruits, infant cereal, and dairy products such
as yogurt, but not infant formula. Infant formula is a substitute, not a complement to breastmilk.
In addition to complementary foods, the W